<a href="https://colab.research.google.com/github/MJMortensonWarwick/AI-DL/blob/main/2_2_resnet-50_and_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction and Transfer Learning with ResNet-50
Here we will use the ResNet-50 model in full (no new training). Let's start by adding a picture of an elephant from the internet (available in the Moodle) 

In [ ]:
from google.colab import files
files.upload()

Saving elephant.jpg to elephant.jpg


{'elephant.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x96\x00\x96\x00\x00\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc0\x00\x11\x08\x06\xaf\x07\x80\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\x01\x00\x02\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x04\x01\x02\x05\x06\x07\x08\t\xff\xc4\x00L\x10\x00\x02\x02\x01\x03\x03\x03\x02\x03\x06\x05\x02\x04\x03\x01\x11\x01\x02\x00\x03\x11\x04\x12!\x051A\x13"Q\x06a\x142q\x07#B\x81\x91\xa1\x08\x15R\xb1\xc13\xd1$br\xe1\x16C\x824S\x92\xa2\xb2\xf0\xf1\x17%&c5T\'sDdt\xff\xc4\x00\x19\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\xff\xc4\x00\'\x11\x01\x01\x01\x01\x00\x03\x00\x02\x02\x03\x01\x01\x01

Here we will call the model, load our image, pre-process it in the fashion ResNet expects and then predict it. Unsuprisingly this is an easy task for ResNet.

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = '/content/elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02504458', 'African_elephant', 0.42291683), ('n02504013', 'Indian_elephant', 0.3574935), ('n01871265', 'tusker', 0.21937707)]


The prediction is "african elephant" which is correct. We can  see "indian elephant" is a close second. 

Let's expand this by instead implementing a transfer learning model. We'll remove the dense neural network layers at the end (FC layers) and learn some new ones for our dataset.

In [ ]:
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras.applications.resnet import preprocess_input, ResNet50
import tensorflow.keras as K 
import tensorflow as tf

def preprocess_res(X, Y):
    X_out = preprocess_input(X)
    Y_out = K.utils.to_categorical(Y, 10)
    return X_out, Y_out

(x_train, y_train), (x_test, y_test) = load_data()
x_train, y_train = preprocess_res(x_train, y_train)
x_test, y_test = preprocess_res(x_test, y_test)

res_model = ResNet50(include_top=False,
                      weights="imagenet",
                      input_tensor=K.Input(shape=(32, 32, 3)))

for layer in res_model.layers[:143]:
    layer.trainable = False

model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.RMSprop(),
                  metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=1,
                        validation_data=(x_test, y_test))
model.summary()

Epoch 1/5
1563/1563 [==============================] - 2075s 1s/step - loss: 1.3963 - accuracy: 0.5406 - val_loss: 1.0369 - val_accuracy: 0.6769
Epoch 2/5
1563/1563 [==============================] - 2101s 1s/step - loss: 1.1189 - accuracy: 0.6401 - val_loss: 0.9158 - val_accuracy: 0.7059
Epoch 3/5
1563/1563 [==============================] - 2122s 1s/step - loss: 1.0187 - accuracy: 0.6732 - val_loss: 0.8589 - val_accuracy: 0.7148
Epoch 4/5
1563/1563 [==============================] - 2123s 1s/step - loss: 0.9374 - accuracy: 0.7001 - val_loss: 0.8458 - val_accuracy: 0.7294
Epoch 5/5
1563/1563 [==============================] - 2105s 1s/step - loss: 0.8824 - accuracy: 0.7218 - val_loss: 0.8511 - val_accuracy: 0.7298
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                       

A lot more to unpack. Firstly, we just run a simple function using the ResNet50 functionality again to pre-process our data. 

Then we again build the ResNet model but this time with an import difference ... _include\_top=False_. This means we have removed the dense (FC) layers that end the model and make predictions. We replace these with our own. 

We also then change the parameter on these ResNet layers to be _trainable=False_. Through doing so, the ResNet part will not change any weights during training, we'll use the ones the author's learned. In other words, the ResNet model is now a feature extractor that preprocesses our data before we use a more vanilla neural net to make predicitons.

Finally, we add layers as we've seen before and compile and run (and wait ... its a big model).

Once trained we can use the model like we would any other:

In [ ]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 44s 139ms/step - loss: 0.8511 - accuracy: 0.7298


[0.8511039018630981, 0.7297999858856201]

All done 🥳